## Import các thư viện cần thiết

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForTokenClassification, AutoModelForSequenceClassification, AutoModelForTokenClassification, Trainer, TrainingArguments, pipeline
import json
import numpy as np
import re
import evaluate
from collections import defaultdict

In [2]:
val_list_path = 'MultiWOZ2.4-main/data/mwz24/MULTIWOZ2.4/valListFile.json'
test_list_path = 'MultiWOZ2.4-main/data/mwz24/MULTIWOZ2.4/testListFile.json'
data_path = 'MultiWOZ2.4-main/data/mwz24/MULTIWOZ2.4/data.json'
dialogue_acts_path = 'MultiWOZ2.4-main/data/mwz24/MULTIWOZ2.4/dialogue_acts.json'
ontology_path = 'MultiWOZ2.4-main/data/mwz24/MULTIWOZ2.4/ontology.json'

## Đọc danh sách mã hội thoại val và test

In [3]:
with open(val_list_path) as f:
    val_list = [line.strip() for line in f]
    
with open(test_list_path) as f:
    test_list = [line.strip() for line in f]

## Đọc dữ liệu hội thoại

In [4]:
with open(data_path) as f:
    data = json.load(f)
    
with open(dialogue_acts_path) as f:
    dialogue_acts = json.load(f)
    
with open(ontology_path) as f:
    ontology = json.load(f)

In [5]:
dialogue_ids = list(data.keys())

train_list = [dialogue_id for dialogue_id in dialogue_ids if dialogue_id not in val_list and dialogue_id not in test_list]

In [6]:
# Lấy tất cả các dialogue acts
acts = set()

for dialogue in dialogue_acts.values():
    for turn in dialogue.values():
        if turn == 'No Annotation':
            continue
        for act in turn.keys():
            acts.add(act)
            
acts.add('No Annotation')
acts = list(acts)

In [7]:
print('Số lượng acts:', len(acts))
print(acts)

Số lượng acts: 32
['Attraction-Request', 'Taxi-Inform', 'Restaurant-Select', 'Train-NoOffer', 'Attraction-Select', 'Train-Select', 'general-bye', 'Booking-Book', 'Hotel-Recommend', 'Restaurant-Inform', 'Restaurant-Recommend', 'Attraction-Inform', 'Train-Request', 'Booking-NoBook', 'Restaurant-NoOffer', 'No Annotation', 'general-greet', 'Train-OfferBook', 'Attraction-NoOffer', 'Hotel-Request', 'Hotel-Inform', 'Hotel-Select', 'Hotel-NoOffer', 'Booking-Request', 'Train-OfferBooked', 'general-welcome', 'Taxi-Request', 'Attraction-Recommend', 'general-reqmore', 'Booking-Inform', 'Train-Inform', 'Restaurant-Request']


In [8]:
# Tạo một từ điển chuyển đổi từ act sang index và ngược lại
act2idx = {act: idx for idx, act in enumerate(acts)}
idx2act = {idx: act for act, idx in act2idx.items()}

In [9]:
# Danh sách các slot dạng phân loại và không phân loại
# Danh sách này được định nghĩa theo ý hiểu cá nhân
# Với biến phân loại, ta sẽ chuyển thành one-hot vector
# Với biến không phân loại, ta sẽ xác định span của nó trong câu

categorical_slots = {
    'attraction-area',
    'attraction-type',
    'bus-day',
    'hotel-area',
    'hotel-internet',
    'hotel-parking',
    'hotel-pricerange',
    'hotel-book day',
    'hotel-stars',
    'hotel-type',
    'restaurant-area',
    'restaurant-book day',
    'restaurant-pricerange',
    'train-departure',
    'train-destination',
    'train-day',
}

non_categorical_slots = {
    'attraction-name',
    'bus-arriveBy',
    'bus-book people',
    'bus-departure',
    'bus-destination',
    'bus-leaveAt',
    'hospital-department',
    'hotel-book people',
    'hotel-book stay',
    'hotel-name',
    'restaurant-book people',
    'restaurant-book time',
    'restaurant-food',
    'restaurant-name',
    'taxi-arriveBy',
    'taxi-departure',
    'taxi-destination',
    'taxi-leaveAt',
    'train-arriveBy',
    'train-book people',
    'train-leaveAt'
}

In [10]:
# Tạo nhãn BIO cho các slot không phân loại
bio_list = ['O']
bio_list.extend([item for slot in non_categorical_slots for item in [f'B-{slot}', f'I-{slot}']])
print('Số lượng nhãn BIO:', len(bio_list))
print(bio_list)

Số lượng nhãn BIO: 43
['O', 'B-restaurant-food', 'I-restaurant-food', 'B-attraction-name', 'I-attraction-name', 'B-hospital-department', 'I-hospital-department', 'B-bus-book people', 'I-bus-book people', 'B-train-leaveAt', 'I-train-leaveAt', 'B-train-arriveBy', 'I-train-arriveBy', 'B-taxi-leaveAt', 'I-taxi-leaveAt', 'B-taxi-destination', 'I-taxi-destination', 'B-restaurant-book people', 'I-restaurant-book people', 'B-bus-departure', 'I-bus-departure', 'B-train-book people', 'I-train-book people', 'B-hotel-book stay', 'I-hotel-book stay', 'B-hotel-name', 'I-hotel-name', 'B-taxi-arriveBy', 'I-taxi-arriveBy', 'B-bus-leaveAt', 'I-bus-leaveAt', 'B-hotel-book people', 'I-hotel-book people', 'B-restaurant-name', 'I-restaurant-name', 'B-taxi-departure', 'I-taxi-departure', 'B-bus-arriveBy', 'I-bus-arriveBy', 'B-bus-destination', 'I-bus-destination', 'B-restaurant-book time', 'I-restaurant-book time']


In [11]:
# Tạo một từ điển chuyển đổi từ nhãn BIO sang index và ngược lại
bio2idx = {bio: idx for idx, bio in enumerate(bio_list)}
idx2bio = {idx: bio for bio, idx in bio2idx.items()}

In [12]:
# Tạo nhãn value cho các slot phân loại
categorical_value_list = []

for slot in categorical_slots:
    for value in ontology[slot]:
        for v in re.split(r'\||>', value):
            categorical_value_list.append(f'{slot}={v}')
            
categorical_value_list = list(set(categorical_value_list))
print('Số lượng nhãn value:', len(categorical_value_list))
print(categorical_value_list)

Số lượng nhãn value: 173
['attraction-type=swimming pool', 'train-day=saturday', 'attraction-area=south', 'train-departure=liverpool', 'restaurant-pricerange=none', 'attraction-type=sports', 'restaurant-book day=thursday', 'restaurant-pricerange=cheap', 'restaurant-area=centre', 'train-day=tuesday', 'hotel-internet=dontcare', 'hotel-area=north', 'train-destination=broxbourne', 'hotel-type=none', 'restaurant-book day=sunday', 'attraction-type=special', 'bus-day=wednesday', 'hotel-area=none', 'attraction-type=college', 'train-destination=huntingdon marriott hotel', 'train-departure=leicester', 'hotel-stars=2', 'hotel-area=south', 'train-departure=brookshite', 'restaurant-area=south', 'train-destination=london', 'hotel-parking=free', 'train-departure=stratford', 'train-departure=huntingdon', 'attraction-type=boating', 'train-departure=east london', 'hotel-book day=tuesday', 'train-departure=panahar', 'restaurant-area=east', 'attraction-type=concert', 'train-departure=birmingham new street

In [13]:
cv2idx = {cv: idx for idx, cv in enumerate(categorical_value_list)}
idx2cv = {idx: cv for cv, idx in cv2idx.items()}

## Dataset

In [14]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [15]:
class MultiWozDataset(Dataset):
    def __init__(self, dialogue_data, acts_data, tokenizer, act2idx, bio2idx, cv2idx, max_turn=-1, dialogue_ids=None):
        self.data = self._process_data(
            dialogue_data, acts_data, max_turn, dialogue_ids)
        self.tokenizer = tokenizer
        self.act2idx = act2idx
        self.bio2idx = bio2idx
        self.cv2idx = cv2idx
        self.problem = 1 # 2, 3

    def _process_data(self, dialogue_data, acts_data, max_turn, dialogue_ids):
        data = []
        for dialogue_id, dialogue in dialogue_data.items():
            if dialogue_ids is not None and dialogue_id not in dialogue_ids:
                continue
            turns = dialogue['log']
            history = []
            for i in range(0, len(turns) - 1, 2):
                user_turn = turns[i]
                history.append(user_turn['text'])

                system_turn = turns[i + 1]
                history.append(system_turn['text'])

                # Lấy act của system
                system_acts = acts_data[dialogue_id[:-5]
                                        ].get(str(i//2 + 1), 'No Annotation')
                if system_acts == 'No Annotation':
                    system_acts = ['No Annotation']
                else:
                    system_acts = list(system_acts.keys())

                # Lấy slot, value của user
                slot_values = []
                for domain, domain_value in system_turn['metadata'].items():
                    for slot, value in domain_value['book'].items():
                        if slot == 'booked':
                            continue
                        if value and value != 'not mentioned':
                            slot_values.append(
                                [f'{domain}-book {slot}', value])
                    for slot, value in domain_value['semi'].items():
                        if value and value != 'not mentioned':
                            slot_values.append([f'{domain}-{slot}', value])

                data.append({
                    'dialogue_id': dialogue_id,
                    'history': history[max(0, i - 2 * max_turn):i] if max_turn > 0 else history[:i],
                    'utterance': user_turn['text'],
                    'system_acts': system_acts,
                    'slot_values': slot_values
                })
        return data
    
    def set_problem(self, problem):
        self.problem = problem

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]

        history_text = '[SEP]'.join(data['history'])
        full_text = f'{history_text}[SEP]{data["utterance"]}' if history_text else data['utterance']

        # Tokenize text
        encoding = self.tokenizer(full_text, return_tensors='pt')

        if self.problem == 1:
            act_labels = torch.zeros(len(self.act2idx))
            for act in data['system_acts']:
                act_labels[self.act2idx[act]] = 1
            return {
                'input_ids': encoding['input_ids'].squeeze(0),
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'labels': act_labels
            }

        if self.problem == 2:
            slot_labels = torch.tensor([self.bio2idx['O']] * len(encoding['input_ids'].squeeze(0)))
            for slot, value in data['slot_values']:
                if slot in non_categorical_slots:
                    for v in re.split(r'\||>', value):
                        start, end = self._get_value_start_end(full_text, v)
                        if start == 0 and end == 0:
                            continue
                        slot_labels[start] = self.bio2idx[f'B-{slot}']
                        slot_labels[start + 1:end + 1] = self.bio2idx[f'I-{slot}']
            # Chuyển label của [SEP] và [CLS] thành -100
            sep_idx = (encoding['input_ids'] == self.tokenizer.sep_token_id).nonzero(as_tuple=True)[1]
            cls_idx = (encoding['input_ids'] == self.tokenizer.cls_token_id).nonzero(as_tuple=True)[1]
            slot_labels[sep_idx] = -100
            slot_labels[cls_idx] = -100
            return {
                'input_ids': encoding['input_ids'].squeeze(0),
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'labels': slot_labels
            }
            
        if self.problem == 3:
            categorical_labels = torch.zeros(len(self.cv2idx))
            for slot, value in data['slot_values']:
                if slot in categorical_slots:
                    for v in re.split(r'\||>', value):
                        categorical_labels[self.cv2idx[f'{slot}={v}']] = 1
            return {
                'input_ids': encoding['input_ids'].squeeze(0),
                'attention_mask': encoding['attention_mask'].squeeze(0),
                'labels': categorical_labels
            }

    def _get_value_start_end(self, text, value):
        tokenized_text = self.tokenizer.tokenize(text, add_special_tokens=True)
        tokenized_value = self.tokenizer.tokenize(value)

        start, end = 0, 0
        for id_v, token_v in enumerate(tokenized_value):
            for id_u, token_u in enumerate(tokenized_text):
                if token_v == token_u:
                    # nếu value được tìm thấy trong text
                    if tokenized_value == tokenized_text[id_u:id_u+len(tokenized_value)]:
                        start, end = id_u, id_u+len(tokenized_value) - 1
                        break
                    # nếu số lượng token còn lại trong text ít hơn số lượng token của value
                    elif len(tokenized_text) - id_u + 1 <= len(tokenized_value):
                        break
        return torch.tensor([start, end])

In [16]:
train_dataset = MultiWozDataset(data, dialogue_acts, tokenizer, act2idx, bio2idx, cv2idx, max_turn=3, dialogue_ids=train_list)
val_dataset = MultiWozDataset(data, dialogue_acts, tokenizer, act2idx, bio2idx, cv2idx, max_turn=3, dialogue_ids=val_list)
test_dataset = MultiWozDataset(data, dialogue_acts, tokenizer, act2idx, bio2idx, cv2idx, max_turn=3, dialogue_ids=test_list)

In [18]:
data_collator = DataCollatorWithPadding(tokenizer)

## Mô hình phát hiện system acts

In [22]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])


def sigmoid(x):
    return 1/(1 + np.exp(-x))


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [20]:
system_acts_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = len(acts) , id2label=idx2act, label2id=act2idx, problem_type='multi_label_classification')

training_args = TrainingArguments(
   output_dir="model/system_acts_model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = Trainer(
   model=system_acts_model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/aircsrv5/miniconda3/envs/imlda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_20906/804699869.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.083200,0.078711,0.968267,0.614339,0.748306,0.521055
2,0.077800,0.074072,0.969763,0.651492,0.738784,0.582649
3,0.073200,0.073164,0.970076,0.658938,0.736848,0.595929


TrainOutput(global_step=10647, training_loss=0.08621127546992427, metrics={'train_runtime': 485.6261, 'train_samples_per_second': 350.751, 'train_steps_per_second': 21.924, 'total_flos': 7296220874484864.0, 'train_loss': 0.08621127546992427, 'epoch': 3.0})

In [22]:
trainer.save_model()

In [23]:
# Thử nghiệm trên tập test
print(trainer.evaluate(test_dataset))

{'eval_loss': 0.07349257171154022, 'eval_accuracy': 0.9703014785675529, 'eval_f1': 0.6602327837051406, 'eval_precision': 0.7386869234943028, 'eval_recall': 0.5968434896975011, 'eval_runtime': 7.3859, 'eval_samples_per_second': 998.122, 'eval_steps_per_second': 62.416, 'epoch': 3.0}


In [17]:
system_acts_classifier = pipeline('text-classification', model='model/system_acts_model', device=0, top_k=None)

In [18]:
sample_text = [
    "I need train reservations from norwich to cambridge",
    "I have 133 trains matching your request. Is there a specific day and time you would like to travel?",
    "I'd like to leave on Monday and arrive by 18:00.",
    "There are 12 trains for the day and time you request.  Would you like to book it now?",
    "Before booking, I would also like to know the travel time, price, and departure time please.",
    "There are 12 trains meeting your needs with the first leaving at 05:16 and the last one leaving at 16:16. Do you want to book one of these? ",
    "No hold off on booking for now.  Can you help me find an attraction called cineworld cinema?",
    "Yes it is a cinema located in the south part of town what information would you like on it?",
    "Yes, that was all I needed. Thank you very much!",
    "Thank you for using our system."
]

history = []
max_turn = 3
threshold = 0.5
for turn in range(0, len(sample_text) - 1, 2):
    history_text = '[SEP]'.join(history[max(0, turn - 2 * max_turn):turn])
    full_text = f'{history_text}[SEP]{sample_text[turn]}' if history_text else sample_text[turn]
    print(sample_text[turn])
    print([out['label'] for out in system_acts_classifier(full_text)[0] if out['score'] > threshold])
    print(sample_text[turn + 1])

I need train reservations from norwich to cambridge
['Train-Request']
I have 133 trains matching your request. Is there a specific day and time you would like to travel?
I'd like to leave on Monday and arrive by 18:00.
['Train-Request']
There are 12 trains for the day and time you request.  Would you like to book it now?
Before booking, I would also like to know the travel time, price, and departure time please.
['Train-Request', 'Train-Inform']
There are 12 trains meeting your needs with the first leaving at 05:16 and the last one leaving at 16:16. Do you want to book one of these? 
No hold off on booking for now.  Can you help me find an attraction called cineworld cinema?
['Attraction-Inform', 'general-reqmore']
Yes it is a cinema located in the south part of town what information would you like on it?
Yes, that was all I needed. Thank you very much!
['general-bye']
Thank you for using our system.


## Mô hình phát hiện slot value

### Với slot phân loại

In [19]:
data_collator_for_cv = DataCollatorWithPadding(tokenizer)

In [20]:
train_dataset.set_problem(3)
val_dataset.set_problem(3)
test_dataset.set_problem(3)

In [23]:
categorical_value_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = len(categorical_value_list), id2label=idx2cv, label2id=cv2idx, problem_type='multi_label_classification')

training_args_cv = TrainingArguments(
   output_dir="model/categorical_value_model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer_cv = Trainer(
   model=categorical_value_model,
   args=training_args_cv,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator_for_cv,
   compute_metrics=compute_metrics,
)

trainer_cv.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/aircsrv5/miniconda3/envs/imlda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_11633/3317102083.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_cv = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.046700,0.052513,0.984724,0.432742,0.825522,0.293227
2,0.033300,0.043674,0.989007,0.659430,0.857729,0.535603
3,0.027900,0.041521,0.990451,0.716170,0.874680,0.606296
4,0.025300,0.041931,0.990739,0.727210,0.876831,0.621208
5,0.023900,0.042428,0.990759,0.729666,0.871303,0.627638


TrainOutput(global_step=17745, training_loss=0.03783340016093917, metrics={'train_runtime': 843.0354, 'train_samples_per_second': 336.747, 'train_steps_per_second': 21.049, 'total_flos': 1.2184700578628496e+16, 'train_loss': 0.03783340016093917, 'epoch': 5.0})

In [24]:
trainer_cv.save_model()

In [25]:
trainer_cv.evaluate(test_dataset)

{'eval_loss': 0.04028211161494255,
 'eval_accuracy': 0.9904842255809359,
 'eval_f1': 0.7133138051592176,
 'eval_precision': 0.8695501929390083,
 'eval_recall': 0.6046697905402699,
 'eval_runtime': 13.4662,
 'eval_samples_per_second': 547.445,
 'eval_steps_per_second': 34.234,
 'epoch': 5.0}

### Với slot không phân loại

In [26]:
train_dataset.set_problem(2)
val_dataset.set_problem(2)
test_dataset.set_problem(2)

In [27]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [28]:
seqeval = evaluate.load("seqeval")

def compute_metrics_tf(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [idx2bio[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [idx2bio[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
non_categorical_value_model = AutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=len(bio_list), id2label=idx2bio, label2id=bio2idx
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args_ncv = TrainingArguments(
    output_dir="model/non_categorical_value_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_ncv = Trainer(
    model=non_categorical_value_model,
    args=training_args_ncv,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_tf,
)

trainer_ncv.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.026200,0.022353,0.841603,0.865539,0.853403,0.992461
2,0.019700,0.020116,0.862204,0.873999,0.868061,0.993115
3,0.015500,0.019766,0.881547,0.908919,0.895024,0.994007
4,0.012200,0.022401,0.887569,0.894519,0.891031,0.993705
5,0.008900,0.023972,0.887765,0.892719,0.890235,0.993597


/home/aircsrv5/miniconda3/envs/imlda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aircsrv5/miniconda3/envs/imlda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=17745, training_loss=0.02148783508573797, metrics={'train_runtime': 960.2125, 'train_samples_per_second': 295.653, 'train_steps_per_second': 18.48, 'total_flos': 1.199016127480728e+16, 'train_loss': 0.02148783508573797, 'epoch': 5.0})

In [31]:
trainer_ncv.save_model()

In [32]:
trainer_ncv.evaluate(test_dataset)

{'eval_loss': 0.018289389088749886,
 'eval_precision': 0.8839788732394366,
 'eval_recall': 0.91565605908635,
 'eval_f1': 0.8995386751466834,
 'eval_accuracy': 0.9945015186900702,
 'eval_runtime': 13.8267,
 'eval_samples_per_second': 533.173,
 'eval_steps_per_second': 33.341,
 'epoch': 5.0}

In [33]:
categorical_value_clf = pipeline('text-classification', model='model/categorical_value_model', device=0, top_k=None)
non_categorical_value_tclf = pipeline('ner', model='model/non_categorical_value_model', device=0)

In [36]:
def slot_vlaue_predict(full_text):
    state = defaultdict(list)
    categorical_value_result = categorical_value_clf(full_text)
    non_categorical_value_result = non_categorical_value_tclf(full_text)
    
    for out in categorical_value_result[0]:
        if out['score'] > 0.5:
            slot, value = out['label'].split('=')
            state[slot].append(value)
            
    current_entity = None
    current_value = ""
    
    for item in non_categorical_value_result:
        entity_type = item['entity'][2:]  # Remove the B- or I- prefix
        if item['entity'].startswith('B-'):
            if current_entity:  # Save the previous entity-value pair if exists
                if current_value.find(':') != -1:
                    current_value = current_value.replace(' ', '')
                state[current_entity].append(current_value)
            current_entity = entity_type
            current_value = item['word']
        elif item['entity'].startswith('I-') and current_entity == entity_type:
            if item['word'].startswith('##'):
                current_value += item['word'][2:]
            else:
                current_value += ' ' + item['word']  # Concatenate words for the same entity

    # Append the last entity-value pair
    if current_entity:
        if current_value.find(':') != -1:
            current_value = current_value.replace(' ', '')
        state[current_entity].append(current_value)
        
    return state

In [37]:
state = {}
max_turn = 3
for turn in range(0, len(sample_text) - 1, 2):
    history_text = '[SEP]'.join(history[max(0, turn - 2 * max_turn):turn])
    full_text = f'{history_text}[SEP]{sample_text[turn]}' if history_text else sample_text[turn]
    print(sample_text[turn])
    state = dict(state | slot_vlaue_predict(full_text))
    print(state)
    print(sample_text[turn + 1])

I need train reservations from norwich to cambridge
{'train-destination': ['cambridge'], 'train-departure': ['norwich']}
I have 133 trains matching your request. Is there a specific day and time you would like to travel?
I'd like to leave on Monday and arrive by 18:00.
{'train-destination': ['cambridge'], 'train-departure': ['norwich'], 'train-day': ['monday'], 'train-arriveBy': ['18:00']}
There are 12 trains for the day and time you request.  Would you like to book it now?
Before booking, I would also like to know the travel time, price, and departure time please.
{'train-destination': ['cambridge'], 'train-departure': ['cambridge'], 'train-day': ['monday'], 'train-arriveBy': ['18:00']}
There are 12 trains meeting your needs with the first leaving at 05:16 and the last one leaving at 16:16. Do you want to book one of these? 
No hold off on booking for now.  Can you help me find an attraction called cineworld cinema?
{'train-destination': ['cambridge'], 'train-departure': ['cambridge']